# This is the how you can pull data from the API

In [ ]:
import datetime as dt
import io
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
from runeq import Config, stream
from scipy import stats
import yaml

In [ ]:
def run_test_requests(params: dict, headers, endpoint, field_name, env='staging', extra=True):
    """
    Run a full suite of tests.
    params
        params for the request
    headers:
        client / user credentials
    endpoint:
        name of endpoint that you are querying from e.g. band_power, probability_symptom
    field_name
        name of the measurement e.g. power, probability
    env
        staging, production, or local
    extra
        runs tests for extra querying expressions e.g. first, last, mean, median
    """

    if env=='staging':
        json_url = f'https://stream-staging.runelabs.io/v1/{endpoint}.json'
        csv_url = f'https://stream-staging.runelabs.io/v1/{endpoint}.csv'

    elif env=='prod':
        json_url = f'https://stream.runelabs.io/v1/{endpoint}.json'
        csv_url = f'https://stream.runelabs.io/v1/{endpoint}.csv'
    else:
        raise ValueError('Unknown envinrontment')

    json_params = params
    availability_params = json_params | {'expression': f'availability({field_name})'}
    response = requests.get(json_url, params=json_params, headers=headers)
    availability_response = requests.get(json_url, params=availability_params, headers=headers)
    print(f'{field_name}: \n', response.json(), '\n')
    print(f'availability({field_name}): \n', availability_response.json(), '\n')

    if extra:
        for exp in ['first', 'mean', 'median', 'last']:
            exp_params = json_params | {'expression': f'{exp}({field_name})'}
            resp = requests.get(json_url, params=exp_params, headers=headers)
            print(f'{exp}({field_name}):\n', resp.json(), '\n')

    csv_params = {k:v for k,v in json_params.items() if k != 'partition_size'}
    availability_csv_params = csv_params | {'expression': f'availability({field_name})'}
    response = requests.get(csv_url, params=csv_params, headers=headers)
    availability_response = requests.get(
        csv_url, params=availability_csv_params, headers=headers)
    print(f'{field_name}: \n', response.text, '\n')
    print(f'availability({field_name}): \n', availability_response.text, '\n')


    if extra:
        for exp in ['first', 'mean', 'median', 'last']:
            exp_params = csv_params | {'expression': f'{exp}({field_name})'}
            resp = requests.get(csv_url, params=exp_params, headers=headers)
            print(f'{exp}({field_name}):\n', resp.text, '\n')

## Local

#### v1/event.json

In [ ]:
params = {
    'patient_id': 'dangerzonepatient', 
    'start_time': '1567566520',
    'end_time': '1567566530',
    'device_id': 'kp0ArnOK', #fake_percept
    'event': 'clinical'
}
response = requests.get('http://localhost:8080/v1/event.json', params=params)

## Staging

In [ ]:
config_path = '/Users/poorv/.rune/api-config-staging.yaml'
with open(config_path) as f:
    credentials = yaml.safe_load(f)
    
headers = {
    'X-Rune-User-Access-Token-Id': credentials['access_token_id'],
    'X-Rune-User-Access-Token-Secret': credentials['access_token_secret']
}

#### v1/band_power

In [ ]:
params = {
    'patient_id': '1ce0cbb08eeb4986af2ee7d965e0f505',
    'start_time': '1583357120',
    'end_time': '1583366377',
    'device_id': 'kp0ArnOK'
}

availability_params = params | {'expression': 'availability(power)'}
response = requests.get('https://stream-staging.runelabs.io/v1/band_power.json',
                        params=params, headers=headers)
availability_response = requests.get('https://stream-staging.runelabs.io/v1/band_power.json', 
                                     params=availability_params, headers=headers)

#### v1/probability_symptom

In [ ]:
params = {
    'device_id': 'kp0ArnOK',
    'symptom': 'tremor',
    'patient_id': '1ce0cbb08eeb4986af2ee7d965e0f505',
    'start_time': '1586210745',
    'end_time': '1586229747',
    'resolution': '3000',
    'partition_size': '288',
    'timestamp': 'datetime',
    'timezone': '-14400',
    'severity': '*'
}
run_test_requests(params, headers, 'probability_symptom', 'probability')

## Production

In [ ]:
prod_config_path = '/Users/poorv/.rune/api-config-production.yaml'
with open(prod_config_path) as f:
    production_credentials = yaml.safe_load(f)
    
prod_headers = {
    'X-Rune-User-Access-Token-Id': production_credentials['access_token_id'],
    'X-Rune-User-Access-Token-Secret': production_credentials['access_token_secret']
}

#### v1/band_power

In [ ]:
prod_params = {
    'patient_id': '9c2ffb54dcab4d54a3dbd05770c2c1c5',
    'start_time': '1583357120',
    'end_time': '1583366377',
    'device_id': '0uuQAj6X'
}
run_test_requests(prod_params, prod_headers, 'band_power', 'power', extra=False, env="prod")

In [ ]:
prod_params = {
    'device_id': '0uuQAj6X',
    'symptom': 'tremor',
    'patient_id': '9c2ffb54dcab4d54a3dbd05770c2c1c5',
    'start_time': '1586210745',
    'end_time': '1586229747',
    'resolution': '3000',
    'partition_size': '288',
    'timestamp': 'datetime',
    'timezone': '-14400',
    'severity': '*'
}
run_test_requests(prod_params, prod_headers, 'probability_symptom', 'probability', env='prod')